# **LOADING PACKAGES**

In [1]:
library(dplyr)
library(lubridate)
library(readr)
library(timeDate)
library(stringr)


Attaching package: ‘dplyr’




The following objects are masked from ‘package:stats’:

    filter, lag




The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union





Attaching package: ‘lubridate’




The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




# **LOADING DATA**

In [2]:
cat("\n====================================\n")
cat("LOADING NYC TAXI DATASET\n")
cat("====================================\n\n")

# Load CSV with specified column types to avoid parsing issues
df <- read_csv('/kaggle/input/taxi-dataset/Taxi Datset.csv', 
               show_col_types = FALSE,  # Suppress column type messages
               col_types = cols(
                 VendorID = col_character(),
                 tpep_pickup_datetime = col_character(),
                 tpep_dropoff_datetime = col_character(),
                 store_and_fwd_flag = col_character(),
                 .default = col_double()
               ))

# Check if there were any parsing problems
if(nrow(problems(df)) > 0) {
  cat("⚠ Parsing issues detected:\n")
  cat(sprintf("  Total problematic rows: %s\n", nrow(problems(df))))
  cat("  First 5 problems:\n")
  print(head(problems(df), 5))
  cat("\n  Note: These issues will be handled during data cleaning\n\n")
} else {
  cat("✓ Data loaded successfully with no parsing issues\n\n")
}

cat("Initial dataset structure:\n")
glimpse(df)
cat(sprintf("\nTotal records loaded: %s\n", format(nrow(df), big.mark=",")))
cat(sprintf("Total columns: %s\n", ncol(df)))

LOADING NYC TAXI DATASET


Warning message:
“One or more parsing issues, call `problems()` on your data frame for details,
e.g.:
  dat <- vroom(...)
  problems(dat)”


⚠ Parsing issues detected:
  Total problematic rows: 5
  First 5 problems:
# A tibble: 5 × 5
       row   col expected   actual    file                                      
     <int> <int> <chr>      <chr>     <chr>                                     
1 11916664     1 18 columns 1 columns /kaggle/input/taxi-dataset/Taxi Datset.csv
2 11916665     2 18 columns 2 columns /kaggle/input/taxi-dataset/Taxi Datset.csv
3 11916666     2 18 columns 2 columns /kaggle/input/taxi-dataset/Taxi Datset.csv
4 11916667     1 18 columns 1 columns /kaggle/input/taxi-dataset/Taxi Datset.csv
5 11916668     1 18 columns 1 columns /kaggle/input/taxi-dataset/Taxi Datset.csv

  Note: These issues will be handled during data cleaning



Initial dataset structure:


Rows: 11,916,667
Columns: 18
$ VendorID              <chr> "1", "1", "1", "1", "2", "2", "2", "2", "2", "1"…
$ tpep_pickup_datetime  <chr> "01/01/2020 12:28:15 AM", "01/01/2020 12:35:39 A…
$ tpep_dropoff_datetime <chr> "01/01/2020 12:33:03 AM", "01/01/2020 12:43:04 A…
$ passenger_count       <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 4, 2, 2, 1, 1, 1, 1, 3, …
$ trip_distance         <dbl> 1.20, 1.20, 0.60, 0.80, 0.00, 0.03, 0.00, 0.00, …
$ RatecodeID            <dbl> 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, …
$ store_and_fwd_flag    <chr> "N", "N", "N", "N", "N", "N", "N", "N", "N", "N"…
$ PULocationID          <dbl> 238, 239, 238, 238, 193, 7, 193, 193, 193, 246, …
$ DOLocationID          <dbl> 239, 238, 238, 151, 193, 193, 193, 193, 193, 48,…
$ payment_type          <dbl> 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, …
$ fare_amount           <dbl> 6.00, 7.00, 6.00, 5.50, 3.50, 2.50, 2.50, 0.01, …
$ extra                 <dbl> 3.0, 3.0, 3.0, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 3.0…
$ mta_tax  


Total records loaded: 11,916,667


Total columns: 18


# **1. DATA TYPE CONVERSION**

In [3]:
# ============================================================================
# 1. DATA TYPE CONVERSION
# ============================================================================
cat('\n=== STEP 1: DATA TYPE CONVERSION ===\n')
cat("Converting columns to appropriate data types...\n")

# Convert dates from string format (MM/DD/YYYY HH:MM:SS AM/PM) to datetime
df$tpep_pickup_datetime <- mdy_hms(df$tpep_pickup_datetime)
df$tpep_dropoff_datetime <- mdy_hms(df$tpep_dropoff_datetime)

# Convert categorical variables to factors for better memory usage and analysis
df$VendorID <- as.factor(df$VendorID)             
df$RatecodeID <- as.factor(df$RatecodeID)          
df$store_and_fwd_flag <- as.factor(toupper(df$store_and_fwd_flag))  
df$payment_type <- as.factor(df$payment_type)

# Convert location and passenger fields to integers
df$PULocationID <- as.integer(df$PULocationID)
df$DOLocationID <- as.integer(df$DOLocationID)
df$passenger_count <- as.integer(df$passenger_count)

cat("\nData types after conversion:\n")
glimpse(df)


=== STEP 1: DATA TYPE CONVERSION ===


Converting columns to appropriate data types...



Data types after conversion:


Rows: 11,916,667
Columns: 18
$ VendorID              <fct> 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 1, …
$ tpep_pickup_datetime  <dttm> 2020-01-01 00:28:15, 2020-01-01 00:35:39, 2020-…
$ tpep_dropoff_datetime <dttm> 2020-01-01 00:33:03, 2020-01-01 00:43:04, 2020-…
$ passenger_count       <int> 1, 1, 1, 1, 1, 1, 1, 1, 4, 2, 2, 1, 1, 1, 1, 3, …
$ trip_distance         <dbl> 1.20, 1.20, 0.60, 0.80, 0.00, 0.03, 0.00, 0.00, …
$ RatecodeID            <fct> 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, …
$ store_and_fwd_flag    <fct> N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, N, …
$ PULocationID          <int> 238, 239, 238, 238, 193, 7, 193, 193, 193, 246, …
$ DOLocationID          <int> 239, 238, 238, 151, 193, 193, 193, 193, 193, 48,…
$ payment_type          <fct> 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, …
$ fare_amount           <dbl> 6.00, 7.00, 6.00, 5.50, 3.50, 2.50, 2.50, 0.01, …
$ extra                 <dbl> 3.0, 3.0, 3.0, 0.5, 0.5, 0.5, 0.5, 0.0, 0.5, 3.0…
$ mta_tax  

# **2. DUPLICATE REMOVAL**

In [4]:
cat("\n--- STEP 2: DUPLICATE REMOVAL ---\n")

initial_rows <- nrow(df)

# Identify duplicates before removing
n_duplicates <- nrow(df) - nrow(distinct(df))
pct_duplicates <- round((n_duplicates / nrow(df)) * 100, 2)

cat(sprintf("Duplicates found: %s (%.2f%% of dataset)\n", 
            format(n_duplicates, big.mark = ","),
            pct_duplicates))

# Remove exact duplicate rows
df <- distinct(df)

cat(sprintf("✓ Removed %s duplicate records\n", format(n_duplicates, big.mark=",")))
cat(sprintf("✓ Remaining records: %s\n", format(nrow(df), big.mark=",")))



--- STEP 2: DUPLICATE REMOVAL ---


Duplicates found: 12,949 (0.11% of dataset)


✓ Removed 12,949 duplicate records


✓ Remaining records: 11,903,718


# **3. MISSING VALUES TREATMENT (NAs)**

In [5]:
cat("\n--- STEP 3: MISSING VALUES TREATMENT (NAs) ---\n")

# Count NAs per column
nas_per_column <- as.data.frame(colSums(is.na(df))) %>% 
  setNames('na_count')

cat("\nMissing values by column:\n")
print(nas_per_column[nas_per_column$na_count > 0, , drop = FALSE])

total_nas <- sum(nas_per_column$na_count)
total_cells <- nrow(df) * ncol(df)
pct_nas <- round((total_nas / total_cells) * 100, 2)

cat(sprintf("\nTotal NAs: %s (%.2f%% of all cells)\n", 
            format(total_nas, big.mark=","), 
            pct_nas))

before_na_removal <- nrow(df)

# Remove all rows with any NA values
df <- df %>% na.omit()

after_na_removal <- nrow(df)

cat(sprintf("✓ Removed %s rows with missing values\n", 
            format(before_na_removal - after_na_removal, big.mark=",")))
cat(sprintf("✓ Remaining records: %s\n", format(nrow(df), big.mark=",")))


--- STEP 3: MISSING VALUES TREATMENT (NAs) ---



Missing values by column:


                      na_count
VendorID                 78499
tpep_pickup_datetime         5
tpep_dropoff_datetime        5
passenger_count          78504
trip_distance                5
RatecodeID               78504
store_and_fwd_flag       78504
PULocationID                 5
DOLocationID                 5
payment_type             78504
fare_amount                  5
extra                        5
mta_tax                      5
tip_amount                   5
tolls_amount                 5
improvement_surcharge        5
total_amount                 5
congestion_surcharge         5



Total NAs: 392,580 (0.18% of all cells)


✓ Removed 78,504 rows with missing values


✓ Remaining records: 11,825,214


# **4. TEMPORAL CONSISTENCY**

In [6]:
cat("\n--- STEP 4: TEMPORAL CONSISTENCY ---\n")

# 4.1: Check for invalid datetime sequences
# Find trips where dropoff time <= pickup time (logically impossible)
df_invalid_dropoff <- df %>% 
  filter(tpep_dropoff_datetime <= tpep_pickup_datetime)

total_invalid <- nrow(df_invalid_dropoff)

cat(sprintf("Invalid datetime sequences found: %s\n", 
            format(total_invalid, big.mark=",")))
cat("(These are trips where dropoff time <= pickup time)\n\n")

# Display 10 sample invalid records
if(total_invalid > 0) {
  cat("Sample of invalid records:\n")
  sample_invalid <- df_invalid_dropoff %>% 
    slice_sample(n = min(10, total_invalid)) %>%
    select(VendorID, tpep_pickup_datetime, tpep_dropoff_datetime, 
           trip_distance, fare_amount)
  print(sample_invalid, n = 10)
  
  # Remove invalid records
  df <- df %>% anti_join(df_invalid_dropoff, by = names(df))
  cat(sprintf("\n✓ Removed %s records with invalid datetime sequences\n", 
              format(total_invalid, big.mark=",")))
}

# 4.2: Create trip duration field and filter unrealistic short trips
cat("\n--- Creating trip duration field ---\n")

df <- df %>% 
  mutate(
    trip_duration = round(as.numeric(
      difftime(
        time1 = tpep_dropoff_datetime, 
        time2 = tpep_pickup_datetime,
        units = 'mins'
      )
    ))
  )

# Find trips with duration <= 1 minute
# These are likely errors or cancelled trips
df_short_duration <- df %>% 
  filter(trip_duration <= 1)

total_short_duration <- nrow(df_short_duration)

cat(sprintf("Trips with duration <= 1 minute found: %s\n", 
            format(total_short_duration, big.mark=",")))
cat("(These may be cancelled trips or recording errors)\n")

# Remove short duration trips
before_short_removal <- nrow(df)
df <- df %>% anti_join(df_short_duration, by = names(df))
after_short_removal <- nrow(df)

cat(sprintf("✓ Removed %s short duration trips\n", 
            format(before_short_removal - after_short_removal, big.mark=",")))
cat(sprintf("✓ Remaining records: %s\n", format(nrow(df), big.mark=",")))



--- STEP 4: TEMPORAL CONSISTENCY ---


Invalid datetime sequences found: 7,531


(These are trips where dropoff time <= pickup time)



Sample of invalid records:
# A tibble: 10 × 5
   VendorID tpep_pickup_datetime tpep_dropoff_datetime trip_distance fare_amount
   <fct>    <dttm>               <dttm>                        <dbl>       <dbl>
 1 1        2020-02-25 18:15:17  2020-02-25 18:15:17               0         2.5
 2 2        2020-01-26 23:15:24  2020-01-26 23:15:24               0        52  
 3 1        2020-01-09 12:52:41  2020-01-09 12:52:41               0         2.5
 4 1        2020-02-02 18:35:30  2020-02-02 18:35:30               0         6  
 5 1        2020-01-13 18:04:08  2020-01-13 18:04:08               0         4.5
 6 1        2020-01-07 11:57:36  2020-01-07 11:57:36               0         0  
 7 1        2020-02-04 21:03:15  2020-02-04 21:03:15               0         4.5
 8 1        2020-02-26 07:59:40  2020-02-26 07:59:40               0         2.5
 9 1        2020-01-25 23:00:06  2020-01-25 23:00:06               0         3  
10 1        2020-01-14 19:01:41  2020-01-14 19:01:41           


--- Creating trip duration field ---


Trips with duration <= 1 minute found: 169,065


(These may be cancelled trips or recording errors)


✓ Removed 169,065 short duration trips


✓ Remaining records: 11,648,618


# **5. TEMPORAL FEATURE ENGINEERING**


In [7]:
cat("\n--- STEP 5: TEMPORAL FEATURE ENGINEERING ---\n")
cat("Creating time-based features for analysis...\n\n")

# Extract year from pickup datetime
df <- df %>%
  mutate(year_pickup = year(tpep_pickup_datetime))

cat("✓ Created: year_pickup (e.g., 2020)\n")

# Add quarter with year (format: 2020.1, 2020.2, etc.)
df <- df %>% 
  mutate(pickup_quarter_label = quarter(tpep_pickup_datetime, type = "year.quarter"))

cat("✓ Created: pickup_quarter_label (e.g., 2020.1 for Q1 2020)\n")

# Add semester with year (1 = Jan-Jun, 2 = Jul-Dec)
df <- df %>% 
  mutate(pickup_year_semester = semester(tpep_pickup_datetime, with_year = TRUE))

cat("✓ Created: pickup_year_semester (e.g., 20201 for first semester)\n")

# Add month abbreviation in uppercase (JAN, FEB, MAR, etc.)
df <- df %>% 
  mutate(pickup_month = toupper(month(tpep_pickup_datetime, label = TRUE)))

cat("✓ Created: pickup_month (e.g., JAN, FEB)\n")

# Add day of week abbreviation in uppercase (MON, TUE, WED, etc.)
df$pickup_weekday <- toupper(wday(df$tpep_pickup_datetime, label = TRUE))

cat("✓ Created: pickup_weekday (e.g., MON, TUE)\n")

# Add AM/PM indicator
df$am_pm <- ifelse(am(df$tpep_pickup_datetime), "AM", "PM")

cat("✓ Created: am_pm (AM for midnight-11:59am, PM for noon-11:59pm)\n")

# Add holiday flag using NYSE holidays as proxy for NYC holidays
# Note: This includes major holidays like New Year's, Martin Luther King Day, etc.
holidays <- as.Date(holidayNYSE(2020))
df$is_holiday <- as.Date(df$tpep_pickup_datetime) %in% holidays

cat("✓ Created: is_holiday (TRUE/FALSE flag for NYSE holidays)\n")

cat(sprintf("\n✓ Total temporal features created: 7\n"))
cat(sprintf("✓ Final dataset dimensions: %s rows × %s columns\n", 
            format(nrow(df), big.mark=","), 
            ncol(df)))


--- STEP 5: TEMPORAL FEATURE ENGINEERING ---


Creating time-based features for analysis...



✓ Created: year_pickup (e.g., 2020)


✓ Created: pickup_quarter_label (e.g., 2020.1 for Q1 2020)


✓ Created: pickup_year_semester (e.g., 20201 for first semester)


...

✓ Created: pickup_weekday (e.g., MON, TUE)


✓ Created: am_pm (AM for midnight-11:59am, PM for noon-11:59pm)


✓ Created: is_holiday (TRUE/FALSE flag for NYSE holidays)



✓ Total temporal features created: 7


✓ Final dataset dimensions: 11,648,618 rows × 26 columns
